In [0]:
!wget https://www.dropbox.com/sh/9pt90m8eqk50gw5/AAAF316c8kyLd5kRNpswdG7Ya?dl=0

--2019-08-21 19:35:10--  https://www.dropbox.com/sh/9pt90m8eqk50gw5/AAAF316c8kyLd5kRNpswdG7Ya?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/9pt90m8eqk50gw5/AAAF316c8kyLd5kRNpswdG7Ya [following]
--2019-08-21 19:35:11--  https://www.dropbox.com/sh/raw/9pt90m8eqk50gw5/AAAF316c8kyLd5kRNpswdG7Ya
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5074790aadca9795c8e8f309ca.dl.dropboxusercontent.com/zip_by_token_key?key=AnEt3oSP9UkFcd7KPpf18duqSqt_zKsX9mkwJbCRgAjqIvh3T6LhBu7yArx4DeRDocr6FjbCPeIe5dFIPdU5i8bidzH46CpwrnaUgc_pkFjPlnASc_3fMk7bKdwrr6g-PEA9Xn3vtDUMBQOTNstOvMGZftLt09JLtA-hYVB8uw-5ji89LBxC89Y69pOe2nyROkqepDJRKJXle6iQiCKPEozh [following]
--2019-08-21 19:35:11--  https://uc5074790aadca9795c8e8f309ca.dl.dropboxuser

In [0]:
!unzip AAAF316c8kyLd5kRNpswdG7Ya?dl=0.2


Archive:  AAAF316c8kyLd5kRNpswdG7Ya?dl=0.2
mapname:  conversion of  failed
replace images/dogs/dog.5.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N

1 archive had fatal errors.


In [0]:
!ls

'AAAF316c8kyLd5kRNpswdG7Ya?dl=0'     images
'AAAF316c8kyLd5kRNpswdG7Ya?dl=0.1'   sample_data


In [0]:
import os
folders=os.listdir('images')
print(folders)
from keras.preprocessing import image

['cats', 'Humans', 'horses', 'dogs']


In [0]:
for f in folders:
  path='images/'+f
  print(f+ " "+str(len(os.listdir(path))))

cats 202
Humans 202
horses 202
dogs 202


In [0]:
import matplotlib.pyplot as plt
import numpy as np

In [0]:
image_data=[]
labels=[]
label_dict={
    "cats":0,
    "dogs":1,
    "horses":2,
    "Humans":3
}
for ix in folders:
  #means making path like images/horses
  path=os.path.join("images",ix)
  for im in os.listdir(path):
    img=image.load_img(os.path.join(path,im),target_size=(224,224))
    img_array=image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])
    
print(len(image_data),len(labels))
##joining image dtata to corresponding labels
combined=list(zip(image_data,labels))
np.random.shuffle(combined)
image_data[:],labels[:]=zip(*combined)    
    
  

808 808


In [0]:
x_train=np.array(image_data)
y_train=np.array(labels)
print(x_train.shape,y_train.shape)

(808, 224, 224, 3) (808,)


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.utils import np_utils
Y_train=np_utils.to_categorical(y_train)
print(Y_train.shape)

(808, 4)


In [0]:
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model

In [0]:
model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation='relu')(av1)
d1=Dropout(0.5)(fc1)
fc2=Dense(4,activation='softmax')(d1)
model_new=Model(input=model.input,output=fc2)
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """


In [0]:
adam=Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])
  

0 <keras.engine.input_layer.InputLayer object at 0x7f92b2a8e7f0>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f92b2a8e668>
2 <keras.layers.convolutional.Conv2D object at 0x7f92b2aa07f0>
3 <keras.layers.normalization.BatchNormalization object at 0x7f92b2ab0358>
4 <keras.layers.core.Activation object at 0x7f92b2ac4630>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f92b308a860>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f92b2a53da0>
7 <keras.layers.convolutional.Conv2D object at 0x7f92b2a0eb00>
8 <keras.layers.normalization.BatchNormalization object at 0x7f92b2953128>
9 <keras.layers.core.Activation object at 0x7f92b2953f98>
10 <keras.layers.convolutional.Conv2D object at 0x7f92b2967d30>
11 <keras.layers.normalization.BatchNormalization object at 0x7f92b297ef28>
12 <keras.layers.core.Activation object at 0x7f92b293fa58>
13 <keras.layers.convolutional.Conv2D object at 0x7f92b28b2cc0>
14 <keras.layers.convolutional.Conv2D object at 0x7f92b281fc18>
15 <keras.lay

In [0]:
for ix in range(169):
  model_new.layers[ix].trainable=False
model_new.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
hist=model_new.fit(x_train,Y_train,shuffle=True,batch_size=16,validation_split=0.2,epochs=5)

Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 19s 30ms/step - loss: 0.4525 - acc: 0.8452 - val_loss: 0.1876 - val_acc: 0.9630
Epoch 2/5
646/646 [==============================] - 7s 11ms/step - loss: 0.1147 - acc: 0.9598 - val_loss: 0.2521 - val_acc: 0.9444
Epoch 3/5
646/646 [==============================] - 7s 11ms/step - loss: 0.1131 - acc: 0.9598 - val_loss: 0.2132 - val_acc: 0.9630
Epoch 4/5
646/646 [==============================] - 7s 11ms/step - loss: 0.0623 - acc: 0.9783 - val_loss: 0.1644 - val_acc: 0.9568
Epoch 5/5
646/646 [==============================] - 7s 11ms/step - loss: 0.0501 - acc: 0.9845 - val_loss: 0.2306 - val_acc: 0.9691
